# PART 2
# TRI NGUYEN (33660812)
### I WORKED AND SUBMITTED ALONE

In [51]:
import pandas as pd
import numpy 
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

### import dataset
credit_train = pd.read_csv("creditdefault_train.csv")
credit_test = pd.read_csv("creditdefault_test.csv")

# check null
# print(credit_test.isnull().sum(), credit_train.isnull().sum())

credit_test.head()

,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23
0,1,120000,2,2,2,26,-1,2,0,0,...,2682,3272,3455,3261,0,1000,1000,1000,0,2000
1,0,90000,2,2,2,34,0,0,0,0,...,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000
2,0,140000,2,3,1,28,0,0,2,0,...,12108,12211,11793,3719,3329,0,432,1000,1000,1000
3,0,20000,1,3,2,35,-2,-2,-2,-2,...,0,0,13007,13912,0,0,0,13007,1122,0
4,0,200000,2,3,2,34,0,0,2,0,...,5535,2513,1828,3731,2306,12,50,300,3738,66


In [52]:
train_x = credit_train.drop(columns=["Y"]).values
train_y = credit_train["Y"].values
test_x = credit_test.drop(columns=["Y"]).values
test_y = credit_test["Y"].values

# scale the data 
scaler = sklearn.preprocessing.StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

# print(train_x)

In [53]:
models = {
    'Random Forest': sklearn.ensemble.RandomForestClassifier(),
    'SVM': sklearn.svm.SVC(probability=True), 
    'KNN': sklearn.neighbors.KNeighborsClassifier()
}

In [55]:
param_grid = {
    'Random Forest': {'n_estimators': [50, 100, 200, 300, 400, 500], 'max_depth': [5, 10, 15, 20, 30, 40, 50]},
    'SVM': {'C': [0.1, 1, 5], 'kernel': ['linear', 'rbf']},
    'KNN': {'n_neighbors': [50, 100, 200, 300, 400]},
}

In [58]:
from sklearn.model_selection import RandomizedSearchCV

best_models = {}
for model_name, model in models.items():
    grid_search = RandomizedSearchCV (
        model, param_grid[model_name], 
        cv=5, 
        scoring='accuracy', 
        n_jobs=-1, 
        verbose=1
    )
    grid_search.fit(train_x, train_y)
    best_models[model_name] = grid_search.best_estimator_
    print(f"{model_name} Best Hyperparameters: {grid_search.best_params_}")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Random Forest Best Hyperparameters: {'n_estimators': 200, 'max_depth': 20}
Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\chris\.conda\envs\newgpu\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


SVM Best Hyperparameters: {'kernel': 'rbf', 'C': 1}
Fitting 5 folds for each of 5 candidates, totalling 25 fits


C:\Users\chris\.conda\envs\newgpu\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


KNN Best Hyperparameters: {'n_neighbors': 50}


In [57]:
from sklearn.model_selection import GridSearchCV

best_models = {}
for model_name, model in models.items():
    random_search = GridSearchCV (
        model, param_grid[model_name], 
        n_iter=40, 
        cv=5, 
        scoring='accuracy', 
        n_jobs=-1, 
        verbose=1
    )
    random_search.fit(train_x, train_y)
    best_models[model_name] = random_search.best_estimator_
    print(f"{model_name} Best Hyperparameters: {random_search.best_params_}")

TypeError: GridSearchCV.__init__() got an unexpected keyword argument 'n_iter'

In [ ]:
best_model = best_models['Random Forest'] 
Y_pred = best_model.predict(X_test)
print(f"Accuracy: {accuracy_score(Y_test, Y_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(Y_test, Y_pred)}")
print(f"Classification Report:\n{classification_report(Y_test, Y_pred)}")